In [1]:
import pickle

with open("../int_to_chars", "rb") as file:
    int_to_chars = pickle.load(file)

chars_to_int = dict()
for i in range(len(int_to_chars)):
    chars_to_int[int_to_chars[i]] = i

In [5]:
import torch
import torch.nn.functional as F
import sys
import os

# Add the root directory to the system path
root_path = os.path.abspath(os.path.join(".."))
if root_path not in sys.path:
    sys.path.append(root_path)

from src.model import Model


HSIZE = 128

model = Model(len(int_to_chars), HSIZE)

In [6]:
with open("../model_weights", "rb") as file:
    model.load_state_dict(torch.load(file))

In [7]:
print(model.gru.bN_u.runningMean)

Parameter containing:
tensor([-2.0621,  2.4257, -1.6067, -2.5220, -0.9118,  3.7980, -2.7124, -2.8190,
        -0.6342, -0.3087,  2.8725, -0.5868, -1.7704,  1.7434,  0.5291, -0.0644,
         3.7296, -0.9913,  0.6398, -1.5069,  3.4801,  1.5825, -0.5438,  0.2660,
        -0.3472,  0.3667, -1.4810, -1.4797,  0.2789, -1.2285, -1.3081, -1.9305,
         2.0595,  1.0548, -1.0194,  0.6544,  1.7655,  1.6903, -1.1825, -0.6807,
        -1.5721, -1.0120,  1.3798,  0.9901,  3.3202,  2.6057, -0.2682,  0.4087,
        -0.0751,  1.4298,  0.7328, -0.5683,  1.1924,  0.4600,  2.7206,  0.5908,
        -0.4468,  0.5231,  0.9235,  0.1345, -2.3082, -0.8481, -1.5328, -1.4315,
        -1.7530, -0.6548,  1.5989, -3.1568,  0.1215, -0.2737, -1.6184, -0.1060,
         1.5683,  0.6983, -3.3968, -1.4742,  3.0821,  0.9807, -0.7706, -2.0512,
        -0.6371,  1.6223,  0.9517, -0.5630, -0.2202, -2.7865,  2.9206, -2.1204,
         0.3583, -0.2072,  0.6396,  0.9509, -0.7005, -0.6255, -0.2610,  1.5482,
        -1.6500,  

In [8]:
N = 30

def tokenize(text):
    if len(text) < N:
        text = "a" * (N-len(text)) + text
    out = torch.zeros(N, dtype=torch.long)
    for i, c in enumerate(text):
        out[i] = chars_to_int[c]

    return out.reshape(1, N)

In [10]:
def generate_char(tokens):
    logits = model.forward(tokens, train=False)
    probs = F.softmax(logits, dim=1)
    generated = torch.multinomial(probs, 1).item()
    new_char = int_to_chars[generated]
    return generated, new_char

In [11]:
def generate_text(text, num):
    tokens = tokenize(text)
    for i in range(num):
        token, new_char = generate_char(tokens)
        text += new_char
        tokens = torch.roll(tokens, -1, 1)
        tokens[0][-1] = token

    print(text)

In [19]:
generate_text("the thing that i hate is", 50)

the thing that i hate is n't maching   
 the eria problemsdow market 
 the


In [14]:
import numpy as np

with open("../data/X_test", "rb") as file:
    seq = np.load(file)

In [15]:
line = seq[10]
text = ""

for c in line:
    text += int_to_chars[c]

text

" n't black monday \n but while "